In [8]:
# Dependencies
import gmaps
import pandas as pd
import numpy as np
import requests
import json
from config import gkey
from geopy import geocoders
import time

# Google API Key
gmaps.configure(api_key=gkey)

In [2]:
# import csv with winery names and country names
# File to Load
wine_csv = "resources/red_and_white_combined.csv"

# Read wine data file and store into Pandas DataFrames
full_df = pd.read_csv(wine_csv)

In [3]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12430 entries, 0 to 12429
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             12430 non-null  object 
 1   Country          12430 non-null  object 
 2   Region           12430 non-null  object 
 3   Winery           12430 non-null  object 
 4   Rating           12430 non-null  float64
 5   NumberOfRatings  12430 non-null  int64  
 6   Price            12430 non-null  float64
 7   Year             12430 non-null  object 
 8   Type             12430 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 874.1+ KB


In [5]:
# pulling columns needed for google API and adding them to new df
df = pd.DataFrame()
df["country"] = full_df["Country"].astype(str)
df["region"] = full_df["Region"].astype(str)
df["winery"] = full_df["Winery"].astype(str)
df["name"] = full_df["Name"].astype(str)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12430 entries, 0 to 12429
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  12430 non-null  object
 1   region   12430 non-null  object
 2   winery   12430 non-null  object
 3   name     12430 non-null  object
dtypes: object(4)
memory usage: 388.6+ KB


In [9]:
# LARGE API DATA PULL - UNCOMMENT TO RUN. BUT UNECESSARY SINCE DATA WAS ADDED TO A CSV TO PREVENT NEED TO REPULL FROM API.

rows = []

# assume null
lat = np.nan
lng = np.nan

for indx, row in df.iterrows():
    # flat dictionary
    new_row = {} 
    
    # Target city
    text = f"{row.winery}"
    country = f"{row.country}"
    wine_name = f"{row.name}"
    print(text)
    
    # Build the endpoint URL
    target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={text}&country={country}&key={gkey}"
    
    try:
        # Run a request to endpoint and convert result to json
        response = requests.get(target_url)
        if(response.status_code == 200):

            geo_data = response.json()

            # Extract latitude and longitude
            if (len(geo_data["results"]) > 0):
                lat = geo_data["results"][0]["geometry"]["location"]["lat"]
                lng = geo_data["results"][0]["geometry"]["location"]["lng"]

    
    except Exception as e:
        print(e)

         # nuke it, everything null
        lat = np.nan
        lng = np.nan
                    
    # flatten data and append
    new_row["name"] = wine_name
    new_row["winery"] = text
    new_row["country"] = country
    new_row["latitude"] = lat
    new_row["longitude"] = lng
    
    rows.append(new_row)
    
    # let api breathe
    time.sleep(3)

Famiglia Castellani
Conte d'Attimis Maniago
Weinbau Uwe Schiefer
Krämer - Straight
Luna Gaia
Schneider
Diel
Von Winning
Feudi Branciforti dei Bordonaro
Bove
Torresella
Domaine Pfister
SIGNÆ
Egon Müller - Scharzhof
Tramin
Félix Solís
Spreitzer
Les Héritiers du Comte Lafon
Rings
Cantina Kaltern
Weninger
Weingut A. Diehl
Wittmann
A. Christmann
Burggarten
Martin Tesch
Knipser
Cantina di Soave
Rings
Schloss Vollrads
Tina Pfaffmann
Weingut Seckinger
Trenz
Langmeil
El Vínculo
Château de Schengen
Marc Brédif
Markus Molitor
Leth
Wittmann
Villa Huesgen
Mezzacorona
Bernhard Huber
Klippenkop
René Muré
Manz
Finca Garabelos
Dr. Heger
Emrich-Schönleber
Louis Latour
Ellermann-Spiegel
Emrich-Schönleber
Epicuro
Escudo Rojo
Calmel & Joseph
Pietracupa
Langmeil
Roberto Lucarelli
Castel Sallegg
Cusumano
Bouchard Père & Fils
Tűzkő
Suavia
Domaine Servin
Bernhard Ott
Quintas do Homem
Weingut Erich & Walter Polz
Hofmann
Weingut Bründlmayer
Burkheimer
Fattoria San Donato
Weingut Pflüger
Prinz von Hessen
Hofmann


Gut Hermannsberg
Kurtatsch (Cortaccia)
Schloss Reinhartshausen
Ansgar Clüsserath
Erwin Sabathi
Château de Millet
Paul Jaboulet Aîné
Marco Felluga
Finca Antigua
Jermann
Clemens Busch
Boeckel
Domaine Pfister
Reichsgraf von Kesselstatt
Tenuta Argentiera
Macià Batle
Food Truck
Domaine du Vieux Télégraphe
Lackner Tinnacher
Roodeberg
Dr. Crusius
Weingut Bründlmayer
Kleine Zalze
Paladin
Trimbach
Ritterhof
Dr. Heger
Bernard Baudry
Bodegas Copaboca
Château de Chamirey
Les Jamelles
Alois Lageder
Miguel Torres
Domaine Belleville
Dr. Von Bassermann-Jordan
Neumeister
Cantina Bolzano / Kellerei Bozen
Montecillo
Chaland Jean-Marie
Terlan (Terlano)
Flagstone
Mouton Cadet
Borgo Molino
Ascheri
Masseria Altemura
Colterenzio (Schreckbichl)
Elena Walch
Muratie
Tasca d'Almerita
Lourensford
Château de Saint Cosme
Kurt Hain
Elena Walch
Manz
Settesoli
Gut Hermannsberg
Schloss Gobelsburg
Fritz Haag
Ravenswood
Dr. Bürklin-Wolf
Cusumano
Tianna Negre
Weingut Abraham
Les Salices
I Capitani
Muratie
Domaine Montrose


Caleo
Friedrich Becker
David Moret-Nomine
Saint Clair
Langmeil
Collavini
Loimer
Frank John
Heger
Weingüter Wegeler
Anura
Famille Perrin
Ciconia
Château de Béru
Schloss Lieser
Livio Felluga
Veramonte
Staatskellerei Zürich
Gini
Domaine Santa Duc
Tormaresca
Nautilus
Diel
Domaine Jean Monnier & Fils
Domaine William Fèvre
Wente
Kracher & Sohm
Greywacke
Coopers Creek
Val de Vid
Schiopetto
Schätzel
Roland Lavantureux
Il Carpino
Baron Philippe de Rothschild
Poggio Graffetta
Tardieu-Laurent
Masciarelli
Antinori
Chanson
Milan Nestarec
Maximin Grünhaus
Attems
Terlan (Terlano)
Paul Mas
J.B. Becker
Leitz
Enzo Boglietti
St. Michael-Eppan
Citra
Weinhaus Gebruder Steffen
Dönnhoff
Wieninger
Alain Geoffroy
Château du Cèdre
Oxford Landing
Van Volxem
Firriato
Wittmann
Diemersdal
St. Michael-Eppan
Château de Chemilly
Markus Molitor
Castelfeder
Weiser-Künstler
Dürnberg
Weingut Weegmüller
Waterkloof
Pascal Bouchard
Ansgar Clüsserath
Val de Vid
Künstler
Feudi Branciforti dei Bordonaro
Gérard Bertrand
Oliver Z

Domaine Drouhin
Michel Thomas & Fils
Domaine William Fèvre
Stellenrust
Knipser
Domaines Astruc
Álvarez y Díez
Torre dei Beati
Salomon Undhof
Terrecarsiche 1939
Oliver Zeter
Von Winning
Georg Mosbacher
Rocca delle Macìe
Seifried Estate
Boschendal
Domaine Naturaliste
Colomé
Leefield Station
Roland Lavantureux
Wittmann
Baglio Oro
Gnarly Head
Poderi dal Nespoli
Montecore
Markus Molitor
Wieninger
Weingüter Wegeler
Markus Molitor
Lanzerac
Colterenzio (Schreckbichl)
Domaine Ricard
Paternoster
Reichsrat von Buhl
Château de la Crée
Franz Keller
Domaine Salmon
Bodegas Nekeas
Reichsrat von Buhl
Vignedileo
Mukhrani
Boschkloof
Wolfberger
Casas del Bosque
Von Winning
Baglio di Pianetto
Mastroberardino
Knipser
Deakin Estate
Weingut Bründlmayer
Domaine Henri Boillot
Konrad
Hugel
Duca di Salaparuta
Torres
Masi
Domaine des Deux Cles
Henri Bourgeois
Georg Breuer
Heinrich Vollmer
Attems
Il Poggio di Gavi
Biscardo
Du Toitskloof
Weingut Loersch
Guerila
Arrogant Frog
Dr. Von Bassermann-Jordan
Mukhrani
Poeira

Lavradores de Feitoria
Avondale
Stina
Henri Bourgeois
Markus Molitor
Domaine des Malandes
Kracher
de Ladoucette
Salwey
Cavalchina
Castelfeder
Catena Zapata
Andeluna
Lenotti
Philippe Chavy
Delaire Graff
Hess Select
Wittmann
Neiss
Michel Torino
La Bastide Blanche
Salvador
Leitz
Monchiero Carbone
Torresella
Trimbach
Beringer
Wittmann
Château Malartic-Lagravière
Reinhard Knebel
Terres Dorées
Paul Cluver
Taylors / Wakefield
St. Urbans-Hof
Stemmari
Château l'Hospitalet
Wittmann
Huerta de Albala
Joh. Jos. Prüm
Schneider
Louis Michel & Fils
Vitis Metamorfosis
Glen Carlou
Von Winning
Domaine des Marronniers
Farnese
Schäfer-Fröhlich
Carl Loewen
Dr. Von Bassermann-Jordan
Kleine Zalze
Panizzi
Neethlingshof Estate
Vergelegen
Paul Mas
Nederburg
Domaine Masson-Blondelet
Joseph Drouhin
Grosset
Von Winning
Alois Lageder
Valdivieso
Klein Constantia
Gerard Tremblay
Elvio Cogno
Velenosi
Bernard Defaix
Redwood Creek
Schneider
Torres
Elephant Hill
Weingut Bründlmayer
Araldica
Brancott Estate
Manincor
Cantin

...

Xavier Vignon
Farnese
Huerta de Albala
Banrock Station
Fèlsina
Clos des Papes (Paul Avril)
Clos Henri Vineyard
Château de Pez
Rust En Vrede
Robert Mondavi
Frontera
Poderi dal Nespoli
Heinrich
Château Boyd-Cantenac
Epicuro
Château Clerc Milon
Tridente
Fanti
Château Latour
Conte di Campiano
Morgenster Estate
Château de Chambert
Frescobaldi
Prunotto
Navarrsotillo
Glen Carlou
Neethlingshof Estate
Baron Philippe de Rothschild
Les Halos de Jupiter
Señorío de Grevol
Château Haut-Bages Libéral
Domaine de la Bégude
Château Lousteauneuf
Craggy Range
Poliziano
Los Frailes
Farnese
Simply Sunshine
Orlando Abrigo
Château La Fleur Morange
Penfolds
Liliac
Lorenz
Creation
Martin Waßmer
Aldeya
Mas del Périé - Fabien Jouves
Chacra
Thunevin
Château Fonréaud
Château Ducru-Beaucaillou
Paul Jaboulet Aîné
Manz
Domaine Bousquet
Château Belgrave
Tenuta le Colonne
Château Purcari
Tresmano - Tr3smano
Friedrich Becker
Black Stallion
Château Prieuré-Lichine
E. Guigal
Allesverloren
Lidio Carraro
Baltasar Gracián
Pla

Château Tour Maillet
Château Etang des Colombes
Elena Walch
Denis Durantou
Herdade da Calada
Paul Mas
Señorio de Los Llanos
Lorusso Michele
Ramón Bilbao
Château Fonroque
Mouton Cadet
Napa Cellars
Paul Hobbs
Viña Pedrosa
Masca del Tacco
Château de Pez
Château Cambon La Pelouse
Torres
Latentia Winery
Domaine Lafage
Salentein
Casas del Bosque
Chateau Ste. Michelle
Casillero del Diablo
Allegrini
La Braccesca
Torrevento
La Gerla
Bodegas Imperiales - Abadía de San Quirce
Tenuta San Guido
Keringer
Andeluna
Gesellmann
Château Fortuneau
Zeni
Salwey
Château de Saint Cosme
Gracia
Swartland Winery
Capanna
Famille Cros Pujol
Château Duhart-Milon
Château de Pez
Johannes Zillinger
Anthonij Rupert
Vietti
Vilafonté
Château Hosanna
Baglio del Cristo di Campobello
Château Péby Faugères
Ziereisen
Bodegas Care
Poggio Al Tesoro
Château Lézin
Château La Cardonne
Hermon
Tinazzi
Sartori
Weingut Metzger
Domaine Ratte
Vinícola del Priorat
Thelema
Virna
El Grifo
Suvla
Tenuta Degli Dei
Peter Falke
Château Labégorc

Domaine Bruno Clair
Pierre Amadieu
Château La Coudraie
Apaltagua
Lar de Maía
Lafran-Veyrolles
Ponte Lungo
Tenute Silvio Nardi
Orin Swift
Torres
Vigneti del Salento
Banfi
Domaine de la Côte
Tolaini
Ramos Pinto
Niepoort
Château Fleur Cardinale
Frescobaldi
Château Lascombes
Miguel Torres
Val d'Orbieu
Château Puech-Haut
Kunde
Cadet d'Oc
Mitolo
Gallo Family Vineyards
Domaine Bousquet
Cono Sur
Cune (CVNE)
Brancaia
Apaltagua
Altos de Rioja
Rocca di Montegrossi
Château Barit
Casa Ermelinda Freitas
Vindemio
Les Vins de Vienne - Cuilleron-Gaillard-Villard
Talenti
Krebs
Château l'Ecuyer
Tardieu-Laurent
Col d'Orcia
Clos Trimoulet
Tedeschi
Costers del Priorat
Château l'Arrosee
Franz Haas
Deakin Estate
M. Chapoutier
Isole e Olena
Schola Sarmenti
Gianfranco Alessandria
Masseria Coste di Brenta
Vindemio
Salentein
Petra
Roc d'Anglade
Vinattieri Ticinesi
O. Fournier
Mas des Mas
Palagetto
Ernst Triebaumer
Château Laforge
Vincenzo Bossotti
Franco-Espanolas
Sileni Estates
Jean-Pierre Moueix
Tinto Pesquera


Château Ducru-Beaucaillou
Vegalfaro
Delas
Cantina Kaltern
Clos de l'Oratoire
Cascina Luisin
Château d'Esclans
Borgogno
Château Combel La Serre
Bodegas Roda
Joseph Drouhin
Boscarelli
Talosa
Doña Paula
Marimar Estate
Tramin
Z1N
Marques de Villanueva
Bodegas Príncipe de Viana
Hacienda Grimon
Alpha Domus
Tormaresca
Louis Latour
Famille Cros Pujol
Domaine de la Tour du Bon
Monteabellon
Weingut R&A Pfaffl
La Braccesca
Peter Lehmann
Grand Bateau
Beringer
Lan
La Rioja Alta
d'Arenberg
Farnese
Castello di Gabbiano
Imperial
Miguel Torres
Ruffino
Allegrini
Cielo e Terra
Cepa 21
Château Sénéjac
Bodegas Faustino
Penfolds
La Bastide Blanche
The Prisoner
Salentein
Château Sociando-Mallet
Domaine de la Janasse
Tinto Negro (TintoNegro)
Château Ducru-Beaucaillou
Volver
Bouchard Père & Fils
Bruno Duchêne
Tenuta dei Sette Cieli
Miolo
El Coto
Quinta da Ponte Pedrinha
Domaine Lafage
San Felice
Pira Luigi
Paolo Cottini
Lenotti
Weinbau Uwe Schiefer
Hensel
Villabella
Château Grand-Puy Ducasse
Château de Saint C

Domaine de l'A
Ceretto
Finca Torremilanos
Goyenechea
Camille Cayran
Villa Cavarena
Mottura
Mukhrani
RedHeads
Neethlingshof Estate
Dievole
Bodega Finca Las Moras
Araldica
Domaine de la Solitude
Alain Voge
Jean Leon
Ciacci Piccolomini d'Aragona
Kollwentz
Elio Grasso
E. Pira & Figli
Château Moulinet-Lasserre
Paolo Manzone
Laibach
Château Carbonnieux
Charles Smith
Quinta das Amoras
Château Charmail
Château Chasse-Spleen
Roodeberg
Marques de Casa Concha
San Silvestro
Portia
Poggio Antico
Château La Louvière
Mandrarossa
Paul Achs
El Grifo
Castello di Morcote
Bodegas Valdemar
Roccapesta
Bodegas Mas Alta
Hans Igler
M. Chapoutier
Budureasca
Château Brane-Cantenac
Caldora
Bernard Baudry
Rocca delle Macìe
Louis Jadot
Winzer Krems
Backsberg
Sei Solo
Domaine de Baron'arques
Famille Lignères - Château La Baronne
Les Bertrand
Viña Bujanda
Capichera
Bodegas Langa
Château La Raze Beauvallet
Bad Boy (Mauvais Garçon)
Domenico Clerico
Marques de Casa Concha
Vigneti del Vulture
Matsu
Andeluna
Torres
Châtea

In [10]:
rows

[{'name': '0',
  'winery': 'Famiglia Castellani',
  'country': 'Italy',
  'latitude': nan,
  'longitude': nan},
 {'name': '1',
  'winery': "Conte d'Attimis Maniago",
  'country': 'Italy',
  'latitude': 46.0096606,
  'longitude': 13.3416919},
 {'name': '2',
  'winery': 'Weinbau Uwe Schiefer',
  'country': 'Austria',
  'latitude': 47.2340461,
  'longitude': 16.3484524},
 {'name': '3',
  'winery': 'Krämer - Straight',
  'country': 'Germany',
  'latitude': 47.2340461,
  'longitude': 16.3484524},
 {'name': '4',
  'winery': 'Luna Gaia',
  'country': 'Italy',
  'latitude': 47.2340461,
  'longitude': 16.3484524},
 {'name': '5',
  'winery': 'Schneider',
  'country': 'Germany',
  'latitude': 47.2340461,
  'longitude': 16.3484524},
 {'name': '6',
  'winery': 'Diel',
  'country': 'Germany',
  'latitude': 47.2340461,
  'longitude': 16.3484524},
 {'name': '7',
  'winery': 'Von Winning',
  'country': 'Germany',
  'latitude': 49.4106226,
  'longitude': 8.1863799},
 {'name': '8',
  'winery': 'Feudi Bra

In [11]:
df_loc = pd.DataFrame(rows)
df_loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12430 entries, 0 to 12429
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       12430 non-null  object 
 1   winery     12430 non-null  object 
 2   country    12430 non-null  object 
 3   latitude   12429 non-null  float64
 4   longitude  12429 non-null  float64
dtypes: float64(2), object(3)
memory usage: 485.7+ KB


In [16]:
df2 = df_loc.dropna()
df2.reset_index(drop=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12429 entries, 1 to 12429
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       12429 non-null  object 
 1   winery     12429 non-null  object 
 2   country    12429 non-null  object 
 3   latitude   12429 non-null  float64
 4   longitude  12429 non-null  float64
dtypes: float64(2), object(3)
memory usage: 582.6+ KB


In [17]:
df2.head()

,name,winery,country,latitude,longitude
1,1,Conte d'Attimis Maniago,Italy,46.009661,13.341692
2,2,Weinbau Uwe Schiefer,Austria,47.234046,16.348452
3,3,Krämer - Straight,Germany,47.234046,16.348452
4,4,Luna Gaia,Italy,47.234046,16.348452
5,5,Schneider,Germany,47.234046,16.348452


In [18]:
df2_to_csv = dict(archive_name='lat_and_long.csv')  
df2.to_csv('lat_and_long.csv', index=False) 